In [2]:
import polars as pl

class CONFIG:
    path_str = "/kaggle/input/jane-street-realtime-marketdata-forecasting/train.parquet"
    target_col = "responder_6"
    lag_cols_original = ["date_id", "symbol_id"] + [f"responder_{idx}" for idx in range(9)]
    lag_cols_rename = { f"responder_{idx}" : f"responder_{idx}_lag_1" for idx in range(9)}
    valid_ratio = 0.01
    start_dt = 0

In [3]:
# Use last 2 parquets
train = pl.scan_parquet(
    CONFIG.path_str
).select(
    pl.int_range(pl.len(), dtype=pl.UInt32).alias("id"),
    pl.all(),
).with_columns(
    (pl.col(CONFIG.target_col)).cast(pl.Int32).alias("label"),
).filter(
    pl.col("date_id").gt(CONFIG.start_dt)
)


In [4]:
lags = train.select(pl.col(CONFIG.lag_cols_original))
lags = lags.rename(CONFIG.lag_cols_rename)
lags = lags.with_columns(
    date_id = pl.col('date_id') + 1,  # lagged by 1 day
)
lags = lags.group_by(["date_id", "symbol_id"], maintain_order=True).last()  # pick up last record of previous date
train = train.join(lags, on=["date_id", "symbol_id"],  how="left")

In [5]:
# 전체 훈련 샘플 수를 "date_id" 열을 선택하고 행을 카운트하여 계산
len_train = train.select(pl.col("date_id")).collect().shape[0]
# 검증 비율에 기반하여 검증에 사용할 레코드 수 결정
valid_records = int(len_train * CONFIG.valid_ratio)
# 오프라인 모델(훈련)에 사용할 레코드 수 계산
len_ofl_mdl = len_train - valid_records
# 계산된 인덱스에서 date_id를 선택하여 오프라인 훈련 세트의 마지막 date_id 가져오기
last_tr_dt = train.select(pl.col("date_id")).collect().row(len_ofl_mdl)[0]
# 전체 훈련 샘플 수 출력
print(f"\n len_train = {len_train}")
# 검증 레코드 수 출력
print(f"\n len_ofl_mdl = {len_ofl_mdl}")
# 마지막 오프라인 훈련 날짜 출력
print(f"\n---> Last offline train date = {last_tr_dt}\n")
training_data = train.filter(pl.col("date_id").le(last_tr_dt))
validation_data   = train.filter(pl.col("date_id").gt(last_tr_dt))

FileNotFoundError: No such file or directory (os error 2): /kaggle/input/jane-street-realtime-marketdata-forecasting/train.parquet

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'select'
	[3] 'with_columns'
	[4] 'filter'
	[5] 'join left'
	[6] 'join'
	[7] 'select'


In [6]:
training_data.collect().\
write_parquet(
    f"/kaggle/input/js24-preprocessing-create-lags/training.parquet", partition_by = "date_id",
)
validation_data.collect().\
write_parquet(
    f"/kaggle/input/js24-preprocessing-create-lags/validation.parquet", partition_by = "date_id",
)

NameError: name 'training_data' is not defined

In [10]:
%reload_ext autoreload

import joblib

# List of feature columns excluding non-feature columns
feature_cols = [col for col in train.columns if col not in ['date_id', 'symbol_id', 'id', 'label']]

# Compute means and standard deviations using Polars
means = train.select(feature_cols).mean().collect().to_dicts()[0]
stds = train.select(feature_cols).std().collect().to_dicts()[0]

data_stats = {'mean': means, 'std': stds}

/tmp/ipykernel_2573/1252290557.py:6: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  feature_cols = [col for col in train.columns if col not in ['date_id', 'symbol_id', 'id', 'label']]


FileNotFoundError: No such file or directory (os error 2): /kaggle/input/jane-street-realtime-marketdata-forecasting/train.parquet

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'select'
	[3] 'with_columns'
	[4] 'filter'
	[5] 'join left'
	[6] 'join'


In [7]:
# Save the stats using joblib
import os
os.makedirs("/kaggle/input/jane-street-data-preprocessing", exist_ok=True)
joblib.dump(data_stats, "/kaggle/input/jane-street-data-preprocessing/data_stats.pkl")

['/kaggle/input/jane-street-data-preprocessing/data_stats.pkl']